# 多値クラス分類のそれぞれの勾配降下法
多値クラスを分類する際に勾配降下法を利用しますが、利用する勾配降下法によって挙動が変わります。
通常の勾配降下法(Gradient descent)、確率的勾配降下法(Stochastic gradient descent)、ミニバッチ確率的勾配降下法(Minibatch SGD)
それぞれをIPythonとMP4とGIFいずれかに出力するコードです。  
動作確認済み環境は以下の通り。

項目|バージョン
---|---
OS|Mac OSX 10.10.5(Yosemite)
Python|3.5.1
matplotlib|1.5.1
numpy|1.11.0
pandas|0.18.0
seaborn|0.7.0
ipython|4.1.2

## 必要なライブラリのインポート
[matplotlib.animation](http://matplotlib.org/api/animation_api.html)はのアニメーション化に必要なパッケージです。  
[IPython.display.HTML](https://ipython.org/ipython-doc/3/api/generated/IPython.display.html#IPython.display.HTML)はアニメーションのIPython内での表示に必要なクラスです。

In [13]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
sns.set(font=['Hiragino Kaku Gothic Pro'])

import textwrap
from IPython.display import HTML

## データ群取得用のメソッドの定義
多値クラス分類するために3種類のデータが収められたcsvを読み込みます。

In [14]:
# データ群の取得
def get_dataset():
    data_labels = ['y1', 'y2', 'x1', 'x2', 'pattern']
    data = pd.read_csv('iris.csv', header=None, names=data_labels)

    # 説明変数は 2つ
    columns = ['x1', 'x2']
    x = data[columns]

    # ラベルを 0, 1の列 * 3 に変換 One-Hot表現に変更
    y = data['pattern']
    y = pd.DataFrame({'setosa': (y == 'setosa').astype(int),
                  'versicolor': (y == 'versicolor').astype(int),
                  'virginica': (y == 'virginica').astype(int)
                  })

    # index と同じ長さの配列を作成し、ランダムにシャッフル
    indexer = np.arange(x.shape[0])
    np.random.seed(123456)
    np.random.shuffle(indexer)
    # x, y を シャッフルされた index の順序に並び替え
    x = x.iloc[indexer, ]
    y = y.iloc[indexer, ]
    
    return x, y, data[columns], data['pattern']


## 勾配降下法の各メソッドとソフトマックス関数の定義
[ここ](http://nbviewer.jupyter.org/gist/mitmul/9283713)と[ここ](http://sinhrks.hatenablog.com/entry/2014/11/24/205305)を大いに参考にしています。

In [15]:
# ソフトマックス関数の定義
def p_y_given_x(x, w, b):
    def softmax(x):
        return (np.exp(x).T / np.sum(np.exp(x), axis=1)).T
    return softmax(np.dot(x, w.T) + b)

# 勾配(gradient)を定義
def grad(x, y, w, b):
    error = p_y_given_x(x, w, b) - y
    w_grad = np.zeros_like(w)
    b_grad = np.zeros_like(b)

    for j in range(w.shape[0]):
        w_grad[j] = (error.iloc[:, j] * x.T).mean(axis=1)
        b_grad[j] = error.iloc[:, j].mean()

    return w_grad, b_grad, np.mean(np.abs(error), axis=0)

# 勾配降下法(gradient descent)を定義
def gd(x, y, w, b, eta=0.1, num=15000):
    for i in range(num):
        # 入力をまとめて処理
        w_grad, b_grad, error = grad(x, y, w, b)
        w -= eta * w_grad
        b -= eta * b_grad
        e = np.sum(np.mean(np.abs(y - p_y_given_x(x, w, b))))
        count = int(i + 1)
        if count % 1000 == 0:
            print("training step:" + str(count))
        if count % 50 == 0:
            yield count, w, b, e

# 確率的勾配降下法(stochastic gradient descent)を定義
def sgd(x, y, w, b, eta=0.1, num=100):
    for i in range(num):
        for index in range(x.shape[0]):
            # 一行ずつ処理
            _x = x.iloc[[index], ]
            _y = y.iloc[[index], ]
            w_grad, b_grad, error = grad(_x, _y, w, b)
            w -= eta * w_grad
            b -= eta * b_grad
            e = np.sum(np.mean(np.abs(y - p_y_given_x(x, w, b))))
            count = int(i * 150 + (index+1))
            if count % 1000 == 0:
                print("training step:" + str(count))
            if count % 50 == 0:
                yield count, w, b, e

# ミニバッチ確率的勾配降下法(mini batch stochastic gradient descent)を定義
def msgd(x, y, w, b, eta=0.1, num=5000, batch_size=50):
    for i in range(num):
        for index in range(0, x.shape[0], batch_size):
            # index 番目のバッチを切り出して処理
            _x = x[index:index + batch_size]
            _y = y[index:index + batch_size]
            w_grad, b_grad, error = grad(_x, _y, w, b)
            w -= eta * w_grad
            b -= eta * b_grad
            e = np.sum(np.mean(np.abs(y - p_y_given_x(x, w, b))))
            count = int(i * 3 + (index / 50) + 1)
            if count % 1000 == 0:
                print("training step:" + str(count))
            if count % 50 == 0:
                yield count, w, b, e



## アニメーション用のコールバックメソッドと各種描画用のメソッドの定義
描画方法や出力方法は他と一緒です。
ただし、勾配法は全て描画してしまうと多大なコマ数になってしまうので、ジェネレータを利用して何回か計算して一定の間隔でグラフ描画するようにしています。

In [16]:
# アニメーション用のコールバックメソッド
def animate(i,gen,bx,line_1,line_2,text):

    i2, w, b, e = next(gen)
    # 回帰直線の y 座標を計算
    by0 = -(w[0, 0] - w[1, 0]) / (w[0, 1] - w[1, 1]) * bx - (b[0] - b[1]) / (w[0, 1] - w[1, 1])
    by1 = -(w[1, 0] - w[2, 0]) / (w[1, 1] - w[2, 1]) * bx - (b[1] - b[2]) / (w[1, 1] - w[2, 1])
    # 回帰直線を生成
    line_1.set_data(bx, by0)
    line_2.set_data(bx, by1)
    # 描画するテキストを生成
    wt = "学習ステップ = {0} times\n".format(i2) + \
         "　　　　　　　 |{0[0]:.2f}, {0[1]:.2f}|\n".format(w[0]) + \
         "　　　重みw  = |{0[0]:.2f}, {0[1]:.2f}|\n".format(w[1]) + \
         "　　　　　　　 |{0[0]:.2f}, {0[1]:.2f}|\n".format(w[2]) + \
         "　 バイアスb = [{0[0]:.2f}, {0[1]:.2f}, {0[2]:.2f}]\n".format(b) + \
         "　 誤差error = {0:.3}".format(e)
    text.set_text(wt)

# データ群の描画
def draw_datas(data_set, label_set, ax, colors = ['red', 'blue', 'green']):
    x1 = data_set.columns[0]
    x2 = data_set.columns[1]
    for (species, group), c in zip(data_set.groupby(label_set), colors):
        ax.scatter(x=group[x1], y=group[x2], color=c)

# 動画作成メソッドの定義
def plot_logreg(x, y, orig_x, orig_y, fitter, title, export_to= "IPython"):
    fig, ax = plt.subplots()
    plt.close()
    # 描画領域の縦、横のサイズを指定
    fig.set_figheight(7.0)
    fig.set_figwidth(12.0)
    # X軸を-1.5〜1.5までに設定
    ax.set_xlim((-0.5, 8.0))
    # Y軸を-1.5〜1.5までに設定
    ax.set_ylim((-0.5, 4.0))

    # 回帰直線描画用の x 座標
    bx = np.arange(x.iloc[:, 0].min(), x.iloc[:, 0].max(), 0.1)

    line_1, = ax.plot([], [], color='gray', linestyle='dashed')
    line_2, = ax.plot([], [], color='gray', linestyle='dashed')
    text = ax.text(0.05, 0.95, "", va='top', transform=ax.transAxes)

    # w, b の初期値を作成
    w = np.random.randint(1,100,(y.shape[1], x.shape[1]))/100
    b = np.random.randint(1,100,y.shape[1])/100
    # 勾配法の関数からジェネレータを生成
    gen = fitter(x, y, w, b)
    ax.set_title(title)
    ax.set_ylim(-0.5, 4.0)
    # データ, 表題を描画
    draw_datas(orig_x, orig_y, ax)


    # アニメーションインスタンス作成
    ani = animation.FuncAnimation(fig = fig, func = animate, fargs = (gen,bx,line_1,line_2,text), frames = 299, interval=60)
    
    if export_to == "IPython":
        return HTML(ani.to_html5_video())
    elif export_to == "MP4":
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        ani.save(title + '.mp4', writer=writer)
    elif export_to == "GIF":
        Writer = animation.writers['imagemagick']
        writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        ani.save(title + '.gif', writer=writer)


## 通常の勾配降下法での描画
学習ステップは15000回です。

In [17]:
x, y, orig_x, orig_y = get_dataset()
plot_logreg(x, y, orig_x, orig_y, gd, 'Gradient descent',export_to="IPython")

training step:1000
training step:2000
training step:3000
training step:4000
training step:5000
training step:6000
training step:7000
training step:8000
training step:9000
training step:10000
training step:11000
training step:12000
training step:13000
training step:14000
training step:15000


## 確率的勾配降下法での描画
学習ステップは同じく15000回です。

In [18]:
x, y, orig_x, orig_y = get_dataset()
plot_logreg(x, y, orig_x, orig_y, sgd, 'Stochastic gradient descent',export_to="IPython")

training step:1000
training step:2000
training step:3000
training step:4000
training step:5000
training step:6000
training step:7000
training step:8000
training step:9000
training step:10000
training step:11000
training step:12000
training step:13000
training step:14000
training step:15000


## ミニバッチ確率的勾配降下法での描画
学習ステップは同じく15000回です。

In [19]:
x, y, orig_x, orig_y = get_dataset()
plot_logreg(x, y, orig_x, orig_y, msgd, 'Minibatch SGD',export_to="IPython")

training step:1000
training step:2000
training step:3000
training step:4000
training step:5000
training step:6000
training step:7000
training step:8000
training step:9000
training step:10000
training step:11000
training step:12000
training step:13000
training step:14000
training step:15000
